In [72]:
import pandas as pd
import numpy as np
import re
from lxml import etree

In [73]:
mlb = pd.read_csv('mlb.csv')
nba = pd.read_csv('nba.csv')
nfl = pd.read_csv('nfl.csv')
nhl = pd.read_csv('nhl.csv')

In [74]:
with open('wikipedia_data.html', 'r', encoding='iso-8859-1') as f:
    html = f.read()

tree = etree.HTML(html)

table  = tree.xpath('//table[@class="wikitable sortable jquery-tablesorter"]')[0]

data = []

for row in table.xpath('.//tr'):

    cells = row.xpath('.//td | .//th')
    
    row = []
    for cell in cells:
        text = cell.xpath("descendant-or-self::*/text()")

        pat1 = r'\[.*\]'
        pat2 = r'[^\x00-\x7F]+'

        res = ''.join([re.sub(pat1 + '|' + pat2, '', str.strip()) for str in text])

        row.append(res)
    data.append(row)

PopTeam = pd.DataFrame(data[1:], columns=data[0])

PopTeam.head(n=53)


,Metropolitan area,Country,Pop.rank,Population(2016 est.),B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,"20,153,634",9,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils,11,Red BullsNew York City FC,
1,Los Angeles,United States,2,"13,310,447",8,RamsChargers,DodgersAngels,LakersClippers,KingsDucks,10,GalaxyLos Angeles FC,
2,San Francisco Bay Area,United States,6,"6,657,982",6,49ersRaiders,GiantsAthletics,Warriors,Sharks,7,Earthquakes,
3,Chicago,United States,3,"9,512,999",5,Bears,CubsWhite Sox,Bulls,Blackhawks,6,Fire,
4,DallasFort Worth,United States,4,"7,233,323",4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,
5,"Washington, D.C.",United States,7,"6,131,977",4,Redskins,Nationals,Wizards,Capitals,5,D.C. United,
6,Philadelphia,United States,8,"6,070,500",4,Eagles,Phillies,76ers,Flyers,5,Union,
7,Boston,United States,12,"4,794,447",4,Patriots,Red Sox,Celtics,Bruins,5,Revolution,
8,MinneapolisSaint Paul,United States,17,"3,551,036",4,Vikings,Twins,Timberwolves,Wild,5,United FC,
9,Denver,United States,20,"2,853,077",4,Broncos,Rockies,Nuggets,Avalanche,5,Rapids,


In [75]:
PopTeam.rename(columns={'Population(2016 est.)' : 'Population'}, inplace=True)

In [76]:
mlb = mlb[mlb['year'] == 2018]
nba = nba[nba['year'] == 2018]
nfl = nfl[nfl['year'] == 2018]
nhl = nhl[nhl['year'] == 2018]

sel_mlb = mlb[['W-L%', 'team']]
sel_nba = nba[['W/L%', 'team']]
sel_nfl = nfl[['W-L%', 'team']]
sel_nhl = nhl[['W','L', 'team']]

In [77]:
filtered_mlb = sel_mlb[~sel_mlb['team'].isin(['Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division'])]
filtered_mlb.rename(columns={"W-L%": "W/L"}, inplace=True)

In [78]:
sel_nba['team'] = sel_nba['team'].str.replace( '(\*\s\((\d+)\))|(\s\((\d+)\))', '', regex=True)

C:\Users\droni\AppData\Local\Temp\ipykernel_8884\2819623865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_nba['team'] = sel_nba['team'].str.replace( '(\*\s\((\d+)\))|(\s\((\d+)\))', '', regex=True)


In [79]:
filtered_nfl = sel_nfl[~sel_nfl['team'].isin(['AFC North', 'AFC South', 'AFC West', 'NFC East','NFC North','NFC South', 'NFC West','AFC East'])]
filtered_nfl.rename(columns={"W-L%": "W/L"}, inplace=True)
filtered_nfl['team'] = filtered_nfl['team'].str.rstrip('*+')
filtered_nfl['W/L'] = filtered_nfl['W/L'].astype(float) 

sel_nba.rename(columns={"W/L%": "W/L"}, inplace=True)
sel_nba['W/L'] = sel_nba['W/L'].astype(float)

C:\Users\droni\AppData\Local\Temp\ipykernel_8884\912635653.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_nfl.rename(columns={"W-L%": "W/L"}, inplace=True)
C:\Users\droni\AppData\Local\Temp\ipykernel_8884\912635653.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_nfl['team'] = filtered_nfl['team'].str.rstrip('*+')
C:\Users\droni\AppData\Local\Temp\ipykernel_8884\912635653.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [80]:
filtered_nhl = sel_nhl[~sel_nhl['team'].isin(['Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division'])]
filtered_nhl['team'] = filtered_nhl['team'].str.rstrip('*')
filtered_nhl['W/L'] = filtered_nhl['W'].astype(int)/(filtered_nhl['W'].astype(int) + filtered_nhl['L'].astype(int))
filtered_nhl.drop(['W', 'L'], axis=1,inplace=True)

C:\Users\droni\AppData\Local\Temp\ipykernel_8884\4031804689.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_nhl['team'] = filtered_nhl['team'].str.rstrip('*')
C:\Users\droni\AppData\Local\Temp\ipykernel_8884\4031804689.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_nhl['W/L'] = filtered_nhl['W'].astype(int)/(filtered_nhl['W'].astype(int) + filtered_nhl['L'].astype(int))
C:\Users\droni\AppData\Local\Temp\ipykernel_8884\4031804689.py:4: SettingWithCopyWarning: 
A value is trying to 

In [81]:
nfl = filtered_nfl
mlb = filtered_mlb
nhl = filtered_nhl
nba = sel_nba

PopTeam = PopTeam[['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']]

In [82]:
Pop4nhl = PopTeam[['Population', 'NHL', 'Metropolitan area']]

In [83]:
Pop4nhl.rename(columns={'NHL':'team'}, inplace=True)
nhl['team'] = nhl['team'].apply(lambda x: x.split()[-1])

Pop4nhl = Pop4nhl[Pop4nhl['team'] != '']

Pop4nhl['team'] = Pop4nhl['team'].apply(lambda x: x.split()[-1])

Pop4nhl['team'] = Pop4nhl['team'].str.findall(r'[A-Z][a-z]*')
Pop4nhl = Pop4nhl.explode('team')

nhl

C:\Users\droni\AppData\Local\Temp\ipykernel_8884\3834479364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pop4nhl.rename(columns={'NHL':'team'}, inplace=True)
C:\Users\droni\AppData\Local\Temp\ipykernel_8884\3834479364.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhl['team'] = nhl['team'].apply(lambda x: x.split()[-1])


,team,W/L
1,Lightning,0.701299
2,Bruins,0.714286
3,Leafs,0.653333
4,Panthers,0.594595
5,Wings,0.434783
6,Canadiens,0.420290
7,Senators,0.394366
8,Sabres,0.357143
10,Capitals,0.653333
11,Penguins,0.618421


In [86]:
merge = pd.merge(Pop4nhl, nhl, on='team', how='right')
# merge[merge.isna()['Metropolitan area']]
# print(merge.head(n=59))
# print(merge[['Metropolitan area', 'W/L']].sort_values(by='Metropolitan area'))
metropolitan_winrate = merge.groupby('Metropolitan area')['W/L'].mean()


Metropolitan area
Boston                    0.714286
Buffalo                   0.357143
Calgary                   0.513889
Chicago                   0.458333
Columbus                  0.600000
DallasFort Worth          0.567568
Denver                    0.589041
Detroit                   0.434783
Edmonton                  0.473684
Las Vegas                 0.680000
Los Angeles               0.622895
MiamiFort Lauderdale      0.594595
MinneapolisSaint Paul     0.633803
Montreal                  0.420290
Nashville                 0.746479
New York City             0.518201
Ottawa                    0.394366
Philadelphia              0.617647
Phoenix                   0.414286
Pittsburgh                0.618421
Raleigh                   0.507042
San Francisco Bay Area    0.625000
St. Louis                 0.578947
Tampa Bay Area            0.701299
Toronto                   0.653333
Vancouver                 0.436620
Washington, D.C.          0.653333
Winnipeg                  0.722222
Na